In [23]:
# data_processing.ipynb

import yfinance as yf
import pandas as pd
import numpy as np
import time
import warnings
import requests

# Suppress warnings for a cleaner notebook output
warnings.filterwarnings('ignore')

# --- DATA ACQUISITION: GET ALL S&P 500 TICKERS ---
def get_sp500_tickers():
    """
    Fetches a list of S&P 500 tickers from Wikipedia.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    table = pd.read_html(response.text)
    df = table[0]
    tickers = df['Symbol'].tolist()
    # Yahoo Finance uses BRK-B for Berkshire Hathaway
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    return tickers

try:
    tickers = get_sp500_tickers()
    print(f"Successfully retrieved {len(tickers)} S&P 500 tickers.")
except Exception as e:
    print(f"Failed to retrieve S&P 500 tickers. Using a hardcoded list. Error: {e}")
    tickers = [
        'AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN', 'NVDA', 'BRK-B', 'META', 'TSLA', 'UNH',
        'LLY', 'JNJ', 'XOM', 'V', 'PG', 'JPM', 'HD', 'MA', 'CVX', 'ABBV', 'PFE', 'AVGO',
        'KO', 'PEP', 'BAC', 'MRK', 'COST', 'WMT', 'MCD', 'TMO', 'CSCO', 'DIS', 'NFLX',
        'ACN', 'DHR', 'ADBE', 'NKE', 'ORCL', 'VZ', 'CRM', 'TXN', 'CMCSA', 'SBUX', 'PM',
        'HON', 'INTC', 'UNP', 'SCHW', 'AXP', 'IBM'
    ]

# Download historical data for the last 5 years
# NOTE: Downloading a large number of tickers may lead to rate limiting.
# If you get errors, you may need to add a delay between downloads.
print("\nDownloading historical stock data...")
data = yf.download(tickers, period="5y", group_by='ticker')
print("Data download complete.")

# Restructure the multi-level column index into a single DataFrame
all_data = pd.DataFrame()
for ticker in tickers:
    if ticker in data.columns:
        stock_df = data[ticker].copy()
        stock_df['Ticker'] = ticker
        all_data = pd.concat([all_data, stock_df])

# Reset the index and prepare the data for feature engineering
all_data.reset_index(inplace=True)
all_data.sort_values(by=['Ticker', 'Date'], inplace=True)
all_data.rename(columns={'index': 'Date'}, inplace=True)


# --- FEATURE ENGINEERING AND ADVANCED PATTERN LABELING ---

def find_cup_and_handle(df, cup_depth_pct=0.2, handle_retracement_pct=0.5, lookback_period=60, uptrend_lookback=250):
    """
    Advanced function to detect a cup and handle pattern.
    This version includes a new condition for a preceding uptrend.
    """
    df = df.copy()
    
    # Calculate rolling statistics over the lookback period
    df['rolling_max'] = df['High'].rolling(window=lookback_period).max().shift(1)
    df['rolling_min'] = df['Low'].rolling(window=lookback_period).min().shift(1)
    
    # Calculate the depth of the potential cup
    cup_depth = df['rolling_max'] - df['rolling_min']
    
    # Condition 1: Check for a "cup" shape
    # The price at the end of the cup should be near the price at the beginning
    is_cup_shape = (df['rolling_max'] - df['Close']) / cup_depth < cup_depth_pct
    
    # Condition 2: Check for a "handle"
    # The handle should be a slight downward or sideways movement
    # and should not retrace more than a certain percentage of the cup's depth.
    is_handle_formed = (df['High'] < df['rolling_max']) & \
                       ((df['rolling_max'] - df['Low']) / cup_depth < handle_retracement_pct)
    
    # Condition 3: Volume confirmation (volume should decrease during cup and handle)
    df['rolling_volume_avg'] = df['Volume'].rolling(window=20).mean().shift(1)
    is_low_volume = (df['Volume'] < df['rolling_volume_avg'])

    # NEW Condition 4: Check for a preceding uptrend
    # We check if the price 250 days ago was significantly lower than the current price
    df['pre_cup_uptrend'] = (df['Close'] > df['Close'].shift(uptrend_lookback) * 1.20)

    # Combine all conditions
    df['cup_and_handle'] = np.where(is_cup_shape & is_handle_formed & is_low_volume & df['pre_cup_uptrend'], 1, 0)
    
    return df['cup_and_handle']


def create_features(df):
    """
    Creates a more robust set of technical indicator features for a single stock dataframe.
    """
    df = df.copy()
    
    # Moving Averages
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # MACD (Moving Average Convergence Divergence)
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # Bollinger Bands
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['std_20'] = df['Close'].rolling(window=20).std()
    df['upper_band'] = df['SMA_20'] + (df['std_20'] * 2)
    df['lower_band'] = df['SMA_20'] - (df['std_20'] * 2)
    
    # Average True Range (ATR)
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['ATR'] = true_range.rolling(window=14).mean()
    
    # Volume features
    df['volume_ratio'] = df['Volume'] / df['Volume'].rolling(window=20).mean()
    df['lagged_return_1d'] = df['Close'].pct_change(periods=1)
    
    # We return only the engineered features
    return df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'SMA_20', 'std_20'], errors='ignore')


all_processed_data = pd.DataFrame()
for ticker in all_data['Ticker'].unique():
    stock_df = all_data[all_data['Ticker'] == ticker].copy()
    
    features_df = create_features(stock_df)
    cup_and_handle_labels = find_cup_and_handle(stock_df)
    
    processed_df = pd.concat([
        stock_df.reset_index(drop=True),
        features_df.reset_index(drop=True),
        cup_and_handle_labels.rename('cup_and_handle').reset_index(drop=True)
    ], axis=1)
    
    processed_df = processed_df.loc[:,~processed_df.columns.duplicated()]
    all_processed_data = pd.concat([all_processed_data, processed_df], ignore_index=True)

all_processed_data.dropna(inplace=True)
all_processed_data = all_processed_data.drop(columns=['Close', 'High', 'Low', 'Open', 'Volume', 'Adj Close'], errors='ignore')

output_file = 'processed_stock_data.csv'
all_processed_data.to_csv(output_file, index=False)
print(f"\nProcessed data saved to {output_file}. Rows: {len(all_processed_data)}")
print("\nFinal processed data dtypes:")
print(all_processed_data.dtypes)


Successfully retrieved 503 S&P 500 tickers.



[*********************100%***********************]  503 of 503 completed


Data download complete.

Processed data saved to processed_stock_data.csv. Rows: 602353

Final processed data dtypes:
Date                datetime64[ns]
Ticker                      object
SMA_10                     float64
SMA_50                     float64
RSI                        float64
MACD                       float64
MACD_signal                float64
upper_band                 float64
lower_band                 float64
ATR                        float64
volume_ratio               float64
lagged_return_1d           float64
cup_and_handle               int32
dtype: object
